In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model  # will be using for plotting trend line
from sklearn.preprocessing import MinMaxScaler # for normalizing data
from sklearn.cluster import KMeans 
pd.set_option('display.max_rows', 1000)
%matplotlib inline
from difflib import SequenceMatcher
import seaborn as sn


In [5]:
data = pd.read_csv('spotify_billboard_final.csv')
data.shape

(148556, 22)

In [10]:
# data.head()

In [7]:
data["pop_artist"].value_counts()


1    93386
0    55170
Name: pop_artist, dtype: int64

In [8]:
data["Top100"].value_counts()

0    143239
1      5317
Name: Top100, dtype: int64

In [11]:
data["artist_already_pop"].value_counts()

0    125369
1     23187
Name: artist_already_pop, dtype: int64

In [15]:
# correlation_matrix = data.corr()
# correlation_matrix = correlation_matrix.drop(correlation_matrix.columns[[range(15)]], axis=1)
# print (correlation_matrix)
# sn.heatmap(correlation_matrix, annot=True)
# plt.show() 